In [1]:
# Code connects to proprietery database (Fanbase) and pulls in object 'workspace'
# 'workspace' contains raw Twitter data from Supermetrics
# Note that raw data from proprietary database is not shared with the public, 
# this script is intended to show the process for feature engineering only.

tweet_dataset = Dataset.get_by_name(workspace, name='MIT Sloan - All Data - with ID - v2')




NameError: name 'Dataset' is not defined

In [ ]:
#Import pandas
import pandas as pd

In [ ]:
#Convert raw twitter data to pandas dataframe
twitter_raw = tweet_dataset.to_pandas_dataframe()

In [ ]:
#Create a new dataframe with simplified columns
#Note that 'PostMessage' contains the contents of a tweet, including text, links, and 
#emojis

twitter2 = twitter_raw[['PostID','Date','Month_Year','VideoFlag','Handle',
'ClientCode','Conference','Sport','PostMessage', 
'Impressions', 'VideoViews','Likes', 'Comments','Shares']]

In [ ]:
#Replace misspelled Main Athletics categories with correct spelling
twitter2["Sport"]=twitter2["Sport"].str.replace('Main Atheltics', 'Main Athletics')

In [ ]:
#Create field for month name and create indicator columns
twitter2['Month_Name']=twitter2['Date'].dt.month_name()
twitter2 = pd.get_dummies(twitter2, columns=['Month_Name'])

In [ ]:
#Create indicator columns for day of week of post
twitter2['Day_of_Week']=twitter2['Date'].dt.day_name()
twitter2 = pd.get_dummies(twitter2, columns=['Day_of_Week'])


In [ ]:
#Create indicator columns to determine sport 
twitter2 = pd.get_dummies(twitter2, columns=['Sport'])

In [ ]:
#Create indicator columns to determine conference of Twitter handle
twitter2 = pd.get_dummies(twitter2, columns=['Conference'])


In [ ]:
#Consolidate indicator columns

#Flag whether a Tweet occurred Friday-Sunday
twitter2.loc[(twitter2['Day_of_Week_Friday'] == 1)
| (twitter2['Day_of_Week_Saturday'] == 1)
| (twitter2['Day_of_Week_Sunday'] == 1), 'Fri_to_Sunday_Flag'] = 1

twitter2.loc[(twitter2['Day_of_Week_Friday'] != 1)
& (twitter2['Day_of_Week_Saturday'] != 1)
& (twitter2['Day_of_Week_Sunday'] != 1), 'Fri_to_Sunday_Flag'] = 0


#Flag whether the Tweet came from a 'Power 5' program
twitter2.loc[(twitter2['Conference_ACC'] == 1)
| (twitter2['Conference_B12'] == 1)
| (twitter2['Conference_B10'] == 1)
| (twitter2['Conference_SEC'] == 1)
| (twitter2['Conference_Pac-12'] == 1)
, 'P5_Conference_Flag'] = 1

twitter2.loc[(twitter2['Conference_ACC'] != 1)
& (twitter2['Conference_B12'] != 1)
& (twitter2['Conference_B10'] != 1)
& (twitter2['Conference_SEC'] != 1)
& (twitter2['Conference_Pac-12'] != 1)
, 'P5_Conference_Flag'] = 0



#Flag whether the sport was an Olympic Sport
#Olympic Sport is deemed any sport that is not football or basketball
twitter2.loc[(twitter2['Sport_Football'] != 1)
& (twitter2['Sport_Women\'s Basketball'] != 1)
& (twitter2['Sport_Men\'s Basketball'] != 1)
& (twitter2['Sport_Main Athletics'] != 1)
, 'Sport_Olympic'] = 1

twitter2.loc[(twitter2['Sport_Football'] == 1)
| (twitter2['Sport_Women\'s Basketball'] == 1)
| (twitter2['Sport_Men\'s Basketball'] == 1)
| (twitter2['Sport_Main Athletics'] == 1)
, 'Sport_Olympic'] = 0



#Flag whether a Tweet occurred in Sept-Oct, Nov-Dec, etc. 

#Month - Sept-Oct
twitter2.loc[(twitter2['Month_Name_September'] == 1)
| (twitter2['Month_Name_October'] == 1)
, 'Month_Sep_Oct'] = 1

twitter2.loc[(twitter2['Month_Name_September'] != 1)
& (twitter2['Month_Name_October'] != 1)
, 'Month_Sep_Oct'] = 0

#Month - Nov-Dec
twitter2.loc[(twitter2['Month_Name_November'] == 1)
| (twitter2['Month_Name_December'] == 1)
, 'Month_Nov_Dec'] = 1

twitter2.loc[(twitter2['Month_Name_November'] != 1)
& (twitter2['Month_Name_December'] != 1)
, 'Month_Nov_Dec'] = 0

#Month - Jan-Feb
twitter2.loc[(twitter2['Month_Name_January'] == 1)
| (twitter2['Month_Name_February'] == 1)
, 'Month_Jan_Feb'] = 1

twitter2.loc[(twitter2['Month_Name_January'] != 1)
& (twitter2['Month_Name_February'] != 1)
, 'Month_Jan_Feb'] = 0

#Month - Mar-Apr
twitter2.loc[(twitter2['Month_Name_March'] == 1)
| (twitter2['Month_Name_April'] == 1)
, 'Month_Mar_Apr'] = 1

twitter2.loc[(twitter2['Month_Name_March'] != 1)
& (twitter2['Month_Name_April'] != 1)
, 'Month_Mar_Apr'] = 0

#Month - May-June
twitter2.loc[(twitter2['Month_Name_May'] == 1)
| (twitter2['Month_Name_June'] == 1)
, 'Month_May_June'] = 1

twitter2.loc[(twitter2['Month_Name_May'] != 1)
& (twitter2['Month_Name_June'] != 1)
, 'Month_May_June'] = 0

#Month - July-Aug
twitter2.loc[(twitter2['Month_Name_July'] == 1)
| (twitter2['Month_Name_August'] == 1)
, 'Month_July_Aug'] = 1

twitter2.loc[(twitter2['Month_Name_July'] != 1)
& (twitter2['Month_Name_August'] != 1)
, 'Month_July_Aug'] = 0


In [ ]:
#Create Share (Retweet) Rate Metric, as well as other engagement metrics
#Share Rate defined as shares / impressions

twitter2['Share_Rate'] = (twitter2['Shares']/twitter2['Impressions'])
twitter2['Comment_Rate'] = (twitter2['Comments']/twitter2['Impressions'])
twitter2['Like_Rate'] = (twitter2['Likes']/twitter2['Impressions'])
twitter2['Total_SCLs'] = twitter2['Shares']+twitter2['Comments']+twitter2['Likes']
twitter2['SCL_Rate']=twitter2['Total_SCLs']/twitter2['Impressions']

In [ ]:
#Create a PostMessage Column that has no hyperlinks. 
import re

#Remove Hyperlinks
twitter2['PostMessage'].astype(str)
twitter2['PostMessage2'] = twitter2['PostMessage']
twitter2['PostMessage2'] =twitter2['PostMessage2'].str.replace('http://\S+|https://\S+', '')
twitter2['PostMessage2'] =twitter2['PostMessage2'].str.replace('http[s]?://\S+', '')
twitter2['PostMessage2'] =twitter2['PostMessage2'].str.replace(r"http\S+", '')



In [ ]:
#Convert PostMessage2 to lower, strip extra white space

twitter2['PostMessage2']= twitter2['PostMessage2'].str.lower()
twitter2['PostMessage2']= twitter2['PostMessage2'].str.strip()


In [ ]:
#Create postmessage fields with varying levels of info (emojis, spaces, etc)

import unicodedata

#Decode unicode text (required for string functions)
twitter2["PostMessage3"] = twitter2["PostMessage2"].str.normalize('NFKD').str.lower().str.encode('ascii', 'ignore')
twitter2["PostMessage4"]=twitter2["PostMessage3"].str.decode('utf8')
twitter2["PostMessage5"]=twitter2["PostMessage4"].astype(str)

#Remove spaces (still contains punctuation)
twitter2["PostMessage6"]=twitter2["PostMessage5"].str.replace('\n\n', ' ')

#Remove punctuation, symbols
twitter2['PostMessage7']= twitter2['PostMessage6'].str.replace(',', '')
twitter2['PostMessage7']= twitter2['PostMessage7'].str.replace('.', '')
twitter2['PostMessage7']= twitter2['PostMessage7'].str.replace('!', '')
twitter2['PostMessage7']= twitter2['PostMessage7'].str.replace('?', '')
twitter2['PostMessage7']= twitter2['PostMessage7'].str.replace('@', '')
twitter2['PostMessage7']= twitter2['PostMessage7'].str.replace('#', '')
twitter2['PostMessage7']= twitter2['PostMessage7'].str.replace('$', '')
twitter2['PostMessage7']= twitter2['PostMessage7'].str.replace('%', '')
twitter2['PostMessage7']= twitter2['PostMessage7'].str.replace('&', '')
twitter2['PostMessage7']= twitter2['PostMessage7'].str.replace('+', '')
twitter2['PostMessage7']= twitter2['PostMessage7'].str.replace('-', '')
twitter2['PostMessage7']= twitter2['PostMessage7'].str.replace('=', '')




In [ ]:
#Create PostMessage column dedicated to keeping hyperlinks to count links
twitter2["PostMessage_H"] = twitter2["PostMessage"].str.normalize('NFKD').str.lower().str.encode('ascii', 'ignore')
twitter2["PostMessage_H"]=twitter2["PostMessage_H"].str.decode('utf8')
twitter2["PostMessage_H"]=twitter2["PostMessage_H"].astype(str)

#Remove spaces -- still contains punctuation
twitter2["PostMessage_H"]=twitter2["PostMessage_H"].str.replace('\n\n', ' ')

#View all versions of 'PostMessage' code
twitter2[["PostMessage", "PostMessage2",
                "PostMessage3", "PostMessage4",
                "PostMessage5", "PostMessage6",
                "PostMessage7",
                "PostMessage_H"]].head()

In [ ]:
#Count exclaimation marks, question marks,
#hashtags, and mentions (Use postmessage field with punct)

twitter2["Exclaim_Mark_Count"]=twitter2["PostMessage6"].map(lambda x: x.count("!"))
twitter2["Question_Mark_Count"]=twitter2["PostMessage6"].map(lambda x: x.count("?"))
twitter2["Hashtag_Count"]=twitter2["PostMessage6"].map(lambda x: x.count("#"))
twitter2["Mention_Count"]=twitter2["PostMessage6"].map(lambda x: x.count("@"))


#Count breaks in tweet (new line -- need to use PostMessage5)
twitter2["Line_Break_Count"]=twitter2["PostMessage5"].map(lambda x: x.count("\n\n"))

#Post Length (number of characters)
twitter2["PostLength_Count"]=twitter2["PostMessage6"].str.len()

#Count hyperlink occurrences
twitter2["Https_Count"]=twitter2["PostMessage_H"].map(lambda x: x.count('https'))
twitter2["Link_Count"]=twitter2["Https_Count"]-1




In [ ]:
import numpy as np
#if link is negative make it '0 links'
twitter2['Link_Count_v2']=np.where(twitter2['Link_Count']<0,0,twitter2['Link_Count'])

In [ ]:
#Count nubmer of emojis used
import emojis

twitter2["Emoji_Count"]= twitter2['PostMessage'].apply(lambda x : emojis.count(str(x)))


In [ ]:
#Generate column to decode emojis into text (':emoji:')
#Decoded emojis can be used to classify whether certain types of emojis
#Show up in Twitter content

twitter2['Emoji_Text'] = twitter2['PostMessage'].apply(lambda x: emojis.decode(str(x)))



In [ ]:
#Create emoji Categories 
#https://www.webfx.com/tools/emoji-cheat-sheet/

#Smiley
emo_smiley_list = [':smile:', ':simple_smile:', ':blush:',':smiley:',':grinning:',':smiling_face:',
':grinning_face:']               
emo_smiley_list_pattern = '|'.join(emo_smiley_list)
twitter2['Emo_Smiley_Flag'] = twitter2['Emoji_Text'].str.contains(emo_smiley_list_pattern).astype(int)

#Heart
emo_heart_list = [':heart:',':red_heart:',':yellow_heart:', ':brown_heart:',':orange_heart:',
':black_heart:',':white_heart:',
':blue_heart:',':purple_heart:',':green_heart:',':heartbeat:',
':heartpulse:',':two_hearts:',':revolving_hearts:',':cupid:',':sparkling_heart:']               
emo_heart_list_pattern = '|'.join(emo_heart_list)
twitter2['Emo_heart_Flag'] = twitter2['Emoji_Text'].str.contains(emo_heart_list_pattern).astype(int)

#Zap
emo_zap_list = [':zap:']               
emo_zap_list_pattern = '|'.join(emo_zap_list)
twitter2['Emo_zap_Flag'] = twitter2['Emoji_Text'].str.contains(emo_zap_list_pattern).astype(int)

#Boom
emo_boom_list = [':boom:',':collision:']               
emo_boom_list_pattern = '|'.join(emo_boom_list)
twitter2['Emo_boom_Flag'] = twitter2['Emoji_Text'].str.contains(emo_boom_list_pattern).astype(int)

#Laugh
emo_laugh_list = [':laughing:',':joy:',':satisfied:']               
emo_laugh_list_pattern = '|'.join(emo_laugh_list)
twitter2['Emo_laugh_Flag'] = twitter2['Emoji_Text'].str.contains(emo_laugh_list_pattern).astype(int)

#Animals
emo_animal_list = [':cat:',':dog:',':mouse:',':hamster:',':rabbit:',':wolf:',
':frog:',':tiger:',':koala:',':bear:',':pig:',':pig_nose:',
':cow:',':boar:',':monkey_face:',':monkey:',':horse:',
':racehorse:',':camel:',':sheep:',':elephant:',':panda_face:',':snake:',
':bird:',':baby_chick:',':hatched_chick:',':hatching_chick:',':chicken:',
':penguin:',':turtle:',':bug:',':honeybee:',':ant:',':beetle:',
':snail:',':octopus:',':tropical_fish:',':fish:',':whale:',':whale2:',
':dolphin:',':cow2:',':ram:',':rat:',':water_buffalo:',':tiger2:',
':rabbit2:',':dragon:',':goat:',':rooster:',':dog2:',':pig2:',':mouse2:',
':ox:',':dragon_face:',':blowfish:',':crocodile:',':dromedary_camel:',
':leopard:',':cat2:',':poodle:',':paw_prints:']
emo_animal_list_pattern = '|'.join(emo_animal_list)
twitter2['Emo_animal_Flag'] = twitter2['Emoji_Text'].str.contains(emo_animal_list_pattern).astype(int)

#Skull
emo_skull_list = [':skull:',':skull_and_crossbones:']               
emo_skull_list_pattern = '|'.join(emo_skull_list)
twitter2['Emo_skull_Flag'] = twitter2['Emoji_Text'].str.contains(emo_skull_list_pattern).astype(int)

#Fire
emo_fire_list = [':fire:']               
emo_fire_list_pattern = '|'.join(emo_fire_list)
twitter2['Emo_fire_Flag'] = twitter2['Emoji_Text'].str.contains(emo_fire_list_pattern).astype(int)

#Dash
emo_dash_list = [':dash:']               
emo_dash_list_pattern = '|'.join(emo_dash_list)
twitter2['Emo_dash_Flag'] = twitter2['Emoji_Text'].str.contains(emo_dash_list_pattern).astype(int)

#Circle
emo_circle_list = [':black_circle:',':white_circle:',':red_circle:',':large_blue_circle:',
                  ':yellow_circle:',':green_circle:',':blue_circle:',':orange_circle:',
                  ':purple_circle:', ':red_circle:',':brown_circle:']               
emo_circle_list_pattern = '|'.join(emo_circle_list)
twitter2['Emo_circle_Flag'] = twitter2['Emoji_Text'].str.contains(emo_circle_list_pattern).astype(int)

#Squares 
emo_Square_list = [':black_small_square:',':white_small_square:',':large_orange_diamond:',
':black_large_square:',':black_medium_small_square:',':white_medium_small_square:',
':white_medium_square:',':black_medium_square:',':white_large_square:',
':white_square_button:',':black_square_button:',':negative_squared_cross_mark:',
':green_square:',':red_square:',':blue_square:',':yellow_square:',':purple_square:',
':black_square:',':white_square:',':orange_square:'
]                
emo_Square_list_pattern = '|'.join(emo_Square_list) 
twitter2['Emo_Square_Flag'] = twitter2['Emoji_Text'].str.contains(emo_Square_list_pattern).astype(int)

#TV and Radio
emo_TV_list = [':tv:',':radio:']                
emo_TV_list_pattern = '|'.join(emo_TV_list) 
twitter2['Emo_TV_Radio_Flag'] = twitter2['Emoji_Text'].str.contains(emo_TV_list_pattern).astype(int)

#Chart
emo_chart_list = [':bar_chart:',':chart_with_upwards_trend:']                
emo_chart_list_pattern = '|'.join(emo_chart_list) 
twitter2['Emo_Bar_chart_Flag'] = twitter2['Emoji_Text'].str.contains(emo_chart_list_pattern).astype(int) 

#Arrows
emo_arrows_list = [':arrow_down:',':arrow_up:',':arrow_heading_down:',
':arrow_heading_up:',':arrows_counterclockwise:',':arrow_double_up',
':arrow_double_down:',':arrow_backward:',':arrow_up_down:',':repeat']                 
emo_arrows_list_pattern = '|'.join(emo_arrows_list)  
twitter2['Emo_arrows_Flag'] = twitter2['Emoji_Text'].str.contains(emo_arrows_list_pattern).astype(int)  

#VS
emo_vs_list = [':vs:']                
emo_vs_list_pattern = '|'.join(emo_vs_list) 
twitter2['Emo_vs_Flag'] = twitter2['Emoji_Text'].str.contains(emo_vs_list_pattern).astype(int) 

#Pushpin
emo_Pushpin_list = [':round_pushpin:']                
emo_Pushpin_list_pattern = '|'.join(emo_Pushpin_list) 
twitter2['Emo_Round_Pushpin_Flag'] = twitter2['Emoji_Text'].str.contains(emo_Pushpin_list_pattern).astype(int) 

#Sports objects
emo_Ball_list = [':football:',':basketball:',':rugby_football:',':baseball:',
':soccer:',':tennis:',':volleyball:',':softball:',':field_hockey:',':ice_hockey:',':lacrosse:',
':golf:',':goal_net:']              
emo_Ball_list_pattern = '|'.join(emo_Ball_list) 
twitter2['Emo_Sports_Objects_Flag'] = twitter2['Emoji_Text'].str.contains(emo_Ball_list_pattern).astype(int) 

#Eyes
emo_eyes_list = [':eyes:']                
emo_eyes_list_pattern = '|'.join(emo_eyes_list) 
twitter2['Emo_eyes_Flag'] = twitter2['Emoji_Text'].str.contains(emo_eyes_list_pattern).astype(int)

#stuck out Eyes
emo_stuck_out_eye_list = [':stuck_out_tongue_winking_eye:',':stuck_out_tongue_closed_eye:']                 
emo_stuck_out_eye_list_pattern = '|'.join(emo_stuck_out_eye_list)  
twitter2['Emo_stuck_out_eye_Flag'] = twitter2['Emoji_Text'].str.contains(emo_stuck_out_eye_list_pattern).astype(int)  

#Rotating light
emo_rot_light_list = [':rotating_light:']                
emo_rot_light_list_pattern = '|'.join(emo_rot_light_list) 
twitter2['Emo_rotating_light_Flag'] = twitter2['Emoji_Text'].str.contains(emo_rot_light_list_pattern).astype(int) 

#Muscle
emo_muscle_list = [':muscle:']                
emo_muscle_list_pattern = '|'.join(emo_muscle_list)
twitter2['Emo_muscle_Flag'] = twitter2['Emoji_Text'].str.contains(emo_muscle_list_pattern).astype(int)

#Alarm Clock
emo_alarm_clock_list = [':alarm_clock:']                
emo_alarm_clock_list_pattern = '|'.join(emo_alarm_clock_list) 
twitter2['Emo_alarm_clock_Flag'] = twitter2['Emoji_Text'].str.contains(emo_alarm_clock_list_pattern).astype(int) 

#Triumph
emo_triumph_list = [':triumph:']                
emo_triumph_list_pattern = '|'.join(emo_triumph_list) 
twitter2['Emo_triumph_Flag'] = twitter2['Emoji_Text'].str.contains(emo_triumph_list_pattern).astype(int) 

#Newspaper
emo_newspaper_list = [':newspaper:',':newspaper_roll:']                
emo_newspaper_list_pattern = '|'.join(emo_newspaper_list) 
twitter2['Emo_newspaper_Flag'] = twitter2['Emoji_Text'].str.contains(emo_newspaper_list_pattern).astype(int)

#Clap
emo_clap_list = [':clap:']                
emo_clap_list_pattern = '|'.join(emo_clap_list) 
twitter2['Emo_clap_Flag'] = twitter2['Emoji_Text'].str.contains(emo_clap_list_pattern).astype(int)

#Metal
emo_metal_list = [':metal:']                
emo_metal_list_pattern = '|'.join(emo_metal_list) 
twitter2['Emo_metal_Flag'] = twitter2['Emoji_Text'].str.contains(emo_metal_list_pattern).astype(int) 

#Computer
emo_computer_list = [':computer:',':desktop_computer:']                
emo_computer_list_pattern = '|'.join(emo_computer_list) 
twitter2['Emo_computer_Flag'] = twitter2['Emoji_Text'].str.contains(emo_computer_list_pattern).astype(int) 

#Link
emo_link_list = [':link:']               
emo_link_list_pattern = '|'.join(emo_link_list)
twitter2['Emo_link_Flag'] = twitter2['Emoji_Text'].str.contains(emo_link_list_pattern).astype(int)

#Check Mark (different colors)
emo_checkmark_list= [':heavy_check_mark:',':white_check_mark:', ':ballot_box_with_check:']
emo_checkmark_list_pattern= '|'.join(emo_checkmark_list)
twitter2['Emo_checkmark_Flag'] = twitter2['Emoji_Text'].str.contains(emo_checkmark_list_pattern).astype(int)

#Heavy_exclaimation_mark, bangbang
emo_heavyexclaimation_list = [':heavy_exclamation_mark:', ':bangbang:', ':interrobang:']               
emo_heavyexclaimation_list_pattern = '|'.join(emo_heavyexclaimation_list)
twitter2['Emo_heavyexclaimation_Flag'] = twitter2['Emoji_Text'].str.contains(emo_heavyexclaimation_list_pattern).astype(int)

#Tickets
emo_ticket_list = [':ticket:']               
emo_ticket_list_pattern = '|'.join(emo_ticket_list)
twitter2['Emo_ticket_Flag'] = twitter2['Emoji_Text'].str.contains(emo_ticket_list_pattern).astype(int)

#Camera flash
emo_camera_list = [':camera_flash:', ':camera:' ]               
emo_camera_list_pattern = '|'.join(emo_camera_list)
twitter2['Emo_camera_Flag'] = twitter2['Emoji_Text'].str.contains(emo_camera_list_pattern).astype(int)

#Triangular flag on post
emo_triflag_list = [':triangular_flag_on_post:']               
emo_triflag_list_pattern = '|'.join(emo_triflag_list)
twitter2['Emo_triflag_Flag'] = twitter2['Emoji_Text'].str.contains(emo_triflag_list_pattern).astype(int)

#Raised hands
emo_raisedhands_list = [':raised_hands:']               
emo_raisedhands_list_pattern = '|'.join(emo_raisedhands_list)
twitter2['Emo_raisedhands_Flag'] = twitter2['Emoji_Text'].str.contains(emo_raisedhands_list_pattern).astype(int)

#Trophy, Ring, Medals
emo_trophyring_list = [':trophy:', ':ring:', ':medal_sports:', ':1st_place_medal:', 
':2nd_place_medal:', ':3rd_place_medal:',':sports_medal:' ]               
emo_trophyring_list_pattern = '|'.join(emo_trophyring_list)
twitter2['Emo_trophyring_Flag'] = twitter2['Emoji_Text'].str.contains(emo_trophyring_list_pattern).astype(int)

#Call Me
emo_callmehand_list = [':call_me_hand:']               
emo_callmehand_list_pattern = '|'.join(emo_callmehand_list)
twitter2['Emo_callmehand_Flag'] = twitter2['Emoji_Text'].str.contains(emo_callmehand_list_pattern).astype(int)

#Handshake
emo_handshake_list = [':handshake:']               
emo_handshake_list_pattern = '|'.join(emo_handshake_list)
twitter2['Emo_handshake_Flag'] = twitter2['Emoji_Text'].str.contains(emo_handshake_list_pattern).astype(int)

#Punch
emo_punch_list = [':punch:', ':facepunch:']               
emo_punch_list_pattern = '|'.join(emo_punch_list)
twitter2['Emo_punch_Flag'] = twitter2['Emoji_Text'].str.contains(emo_punch_list_pattern).astype(int)

#Star Struck
emo_starstruck_list = [':star_struck:']               
emo_starstruck_list_pattern = '|'.join(emo_starstruck_list)
twitter2['Emo_starstruck_Flag'] = twitter2['Emoji_Text'].str.contains(emo_starstruck_list_pattern).astype(int)

#Sunglasses
emo_sunglasses_list = [':sunglasses:']               
emo_sunglasses_list_pattern = '|'.join(emo_sunglasses_list)
twitter2['Emo_sunglasses_Flag'] = twitter2['Emoji_Text'].str.contains(emo_sunglasses_list_pattern).astype(int)

#Pointing
emo_point_list = [':point_up:', ':point_down:', ':point_left:', ':point_right:', ':point_up_2:']               
emo_point_list_pattern = '|'.join(emo_point_list)
twitter2['Emo_point_Flag'] = twitter2['Emoji_Text'].str.contains(emo_point_list_pattern).astype(int)

#Movie
emo_movie_list = [':movie_camera:', ':clapper:', ':cinema:']               
emo_movie_list_pattern = '|'.join(emo_movie_list)
twitter2['Emo_movie_Flag'] = twitter2['Emoji_Text'].str.contains(emo_movie_list_pattern).astype(int)

#OK hand
emo_okhand_list = [':ok_hand:']               
emo_okhand_list_pattern = '|'.join(emo_okhand_list)
twitter2['Emo_okhand_Flag'] = twitter2['Emoji_Text'].str.contains(emo_okhand_list_pattern).astype(int)

#Numbers
emo_numbers_list = [':one:', ':two:', ':three:', ':four:', ':five:',
 ':six:', ':seven:', ':eight:', ':nine:', ':keycap_ten:', ':1234:', ':zero:']               
emo_numbers_list_pattern = '|'.join(emo_numbers_list)
twitter2['Emo_numbers_Flag'] = twitter2['Emoji_Text'].str.contains(emo_numbers_list_pattern).astype(int)

#GOAT
emo_goat_list = [':goat:']               
emo_goat_list_pattern = '|'.join(emo_goat_list)
twitter2['Emo_goat_Flag'] = twitter2['Emoji_Text'].str.contains(emo_goat_list_pattern).astype(int)

#Imp
emo_imp_list = [':imp:', ':smiling_imp:' ]               
emo_imp_list_pattern = '|'.join(emo_imp_list)
twitter2['Emo_imp_Flag'] = twitter2['Emoji_Text'].str.contains(emo_imp_list_pattern).astype(int)

#Weather
emo_weather_list = [':sunny:', ':umbrella:', ':cloud:', ':snowflake:', ':snowman:']               
emo_weather_list_pattern = '|'.join(emo_weather_list)
twitter2['Emo_weather_Flag'] = twitter2['Emoji_Text'].str.contains(emo_weather_list_pattern).astype(int)

#USA Flag
emo_usflag_list = [':us:']               
emo_usflag_list_pattern = '|'.join(emo_usflag_list)
twitter2['Emo_usflag_Flag'] = twitter2['Emoji_Text'].str.contains(emo_usflag_list_pattern).astype(int)

#Thumbs up
emo_thumbsup_list = [':thumbsup:']               
emo_thumbsup_list_pattern = '|'.join(emo_thumbsup_list)
twitter2['Emo_thumbsup_Flag'] = twitter2['Emoji_Text'].str.contains(emo_thumbsup_list_pattern).astype(int)

#Thumbs down
emo_thumbsdown_list = [':thumbsdown:']               
emo_thumbsdown_list_pattern = '|'.join(emo_thumbsdown_list)
twitter2['Emo_thumbsdown_Flag'] = twitter2['Emoji_Text'].str.contains(emo_thumbsdown_list_pattern).astype(int)

#Peace (V) Sign
emo_peace_list = [':v:']               
emo_peace_list_pattern = '|'.join(emo_peace_list)
twitter2['Emo_peacehand_Flag'] = twitter2['Emoji_Text'].str.contains(emo_peace_list_pattern).astype(int)

#Graduation Cap
emo_gradcap_list = [':mortar_board:']               
emo_gradcap_list_pattern = '|'.join(emo_gradcap_list)
twitter2['Emo_gradcap_Flag'] = twitter2['Emoji_Text'].str.contains(emo_gradcap_list_pattern).astype(int)

#Celebration
emo_celebration_list = [':tada:', ':confetti_ball:' ]               
emo_celebration_list_pattern = '|'.join(emo_celebration_list)
twitter2['Emo_celebration_Flag'] = twitter2['Emoji_Text'].str.contains(emo_celebration_list_pattern).astype(int)

#Noise
emo_noise_list = [':speaker:', ':loudspeaker:', ':mega:']               
emo_noise_list_pattern = '|'.join(emo_noise_list)
twitter2['Emo_noise_Flag'] = twitter2['Emoji_Text'].str.contains(emo_noise_list_pattern).astype(int)

#xmas
emo_xmas_list = [':santa:', ':christmas_tree:', ':gift:']               
emo_xmas_list_pattern = '|'.join(emo_xmas_list)
twitter2['Emo_xmas_Flag'] = twitter2['Emoji_Text'].str.contains(emo_xmas_list_pattern).astype(int)

#Halloween
emo_halloween_list = [':jack_o_lantern:', ':ghost:']               
emo_halloween_list_pattern = '|'.join(emo_halloween_list)
twitter2['Emo_halloween_Flag'] = twitter2['Emoji_Text'].str.contains(emo_halloween_list_pattern).astype(int)

#Plants
emo_plants_list = [':cherry_blossom:', ':tulip:', ':four_leaf_clover:',
 ':rose:', ':sunflower:', ':hibiscus:', ':maple_leaf:', ':leaves:', 
 ':fallen_leaf:', ':herb:', ':mushroom:', ':cactus:', ':palm_tree:',
  ':evergreen_tree:', ':deciduous_tree:', ':chestnut:', ':seedling:',
   ':blossom:', ':ear_of_rice:']               
emo_plants_list_pattern = '|'.join(emo_plants_list)
twitter2['Emo_plants_Flag'] = twitter2['Emoji_Text'].str.contains(emo_plants_list_pattern).astype(int)


#Airplane
emo_airplane_list = [':airplane:']               
emo_airplane_list_pattern = '|'.join(emo_airplane_list)
twitter2['Emo_airplane_Flag'] = twitter2['Emoji_Text'].str.contains(emo_airplane_list_pattern).astype(int)

#House
emo_house_list = [':house:', ':house_with_garden:']               
emo_house_list_pattern = '|'.join(emo_house_list)
twitter2['Emo_house_Flag'] = twitter2['Emoji_Text'].str.contains(emo_house_list_pattern).astype(int)

#Calendar
emo_calendar_list = [':calendar:', ':date:']               
emo_calendar_list_pattern = '|'.join(emo_calendar_list)
twitter2['Emo_calendar_Flag'] = twitter2['Emoji_Text'].str.contains(emo_calendar_list_pattern).astype(int)

#Money
emo_money_list = [':moneybag:', ':dollar:', ':money_with_wings:']               
emo_money_list_pattern = '|'.join(emo_money_list)
twitter2['Emo_money_Flag'] = twitter2['Emoji_Text'].str.contains(emo_money_list_pattern).astype(int)

#Hour glass
emo_hourglass_list = [':hourglass:', ':hourglass_flowing_sand:']               
emo_hourglass_list_pattern = '|'.join(emo_hourglass_list)
twitter2['Emo_hourglass_Flag'] = twitter2['Emoji_Text'].str.contains(emo_hourglass_list_pattern).astype(int)

#Exploding Head
emo_exploding_head_list = [':exploding_head:']
emo_exploding_head_list_pattern = '|'.join(emo_exploding_head_list)
twitter2['Emo_exploding_head_Flag'] = twitter2['Emoji_Text'].str.contains(emo_exploding_head_list_pattern).astype(int)

#Wave
emo_wave_list = [':wave:']
emo_wave_list_pattern = '|'.join(emo_wave_list)
twitter2['Emo_wave_Flag'] = twitter2['Emoji_Text'].str.contains(emo_wave_list_pattern).astype(int)

#Speech and thought bubbles
emo_speech_thought_bubble_list = [':speech_balloon:',':thought_balloon:']
emo_speech_thought_bubble_list_pattern = '|'.join(emo_speech_thought_bubble_list)
twitter2['Emo_speech_thought_bubble_Flag'] = twitter2['Emoji_Text'].str.contains(emo_speech_thought_bubble_list_pattern).astype(int)

#Stars
emo_star_list = [':sparkles:', ':star:', ':star2:']
emo_star_list_pattern = '|'.join(emo_star_list)
twitter2['Emo_star_Flag'] = twitter2['Emoji_Text'].str.contains(emo_star_list_pattern).astype(int)

#Lock
emo_lock_list = [':unlock:', ':lock:', ':closed_lock_with_key:']
emo_lock_list_pattern = '|'.join(emo_lock_list)
twitter2['Emo_lock_Flag'] = twitter2['Emoji_Text'].str.contains(emo_lock_list_pattern).astype(int)

#Books
emo_book_list = [':books:', ':green_book:', ':blue_book:', ':orange_book:',
':red_book:', ':open_book:',
':notebook:', ':notebook_with_decorative_cover:', ':ledger:']
emo_book_list_pattern = '|'.join(emo_book_list)
twitter2['Emo_book_Flag'] = twitter2['Emoji_Text'].str.contains(emo_book_list_pattern).astype(int)

#Scream
emo_scream_list = [':scream:']
emo_scream_list_pattern = '|'.join(emo_scream_list)
twitter2['Emo_scream_Flag'] = twitter2['Emoji_Text'].str.contains(emo_scream_list_pattern).astype(int)

#Intellectual Property
emo_intellprop_list = [':copyright:', ':registered:', ':tm:']
emo_intellprop_list_pattern = '|'.join(emo_intellprop_list)
twitter2['Emo_intellprop_Flag'] = twitter2['Emoji_Text'].str.contains(emo_intellprop_list_pattern).astype(int)

#Hot/Sweaty Face
emo_hot_face_list = [':hot_face:']
emo_hot_face_list_pattern = '|'.join(emo_hot_face_list)
twitter2['Emo_hot_face_Flag'] = twitter2['Emoji_Text'].str.contains(emo_hot_face_list_pattern).astype(int)

#Signle Eye
emo_single_eye_list = [':eye:']
emo_single_eye_list_pattern = '|'.join(emo_single_eye_list)
twitter2['Emo_single_eye_Flag'] = twitter2['Emoji_Text'].str.contains(emo_single_eye_list_pattern).astype(int)

#Shout
emo_shout_list = [':speaking_head:']
emo_shout_list_pattern = '|'.join(emo_shout_list)
twitter2['Emo_shout_Flag'] = twitter2['Emoji_Text'].str.contains(emo_shout_list_pattern).astype(int)

#Consolidate all hand gestures
emo_handgesture_list = [':thumbsup:',':clap:',':metal:',':raised_hands:',':call_me_hand:',':handshake:',
':punch:', ':facepunch:',':wave:',':point_up:', ':point_down:', ':point_left:', ':point_right:', ':point_up_2:',
':ok_hand:']
emo_handgesture_list_pattern = '|'.join(emo_handgesture_list)
twitter2['Emo_handgesture_Flag'] = twitter2['Emoji_Text'].str.contains(emo_handgesture_list_pattern).astype(int)




In [ ]:
#Import packages for sentiment analysis
import nltk

from nltk.sentiment.vader import SentimentIntensityAnalyzer
sid = SentimentIntensityAnalyzer()

In [ ]:
#Create sentiment scores using VADER lexicon (pos, neg, and neutral)

#Create a column that contains all sentiment polarity scores (pos, neg, neu)

twitter2['SentScore'] = twitter2['PostMessage6'].apply(lambda x: sid.polarity_scores(x))


#Create additional columns with only pos, neg, and neutral sentiment score results

twitter2['SentScore_NEG'] = (twitter2['SentScore'].astype(str)
                                .str.split(",", n=1, expand=True)
                                .loc[:,0]
                                .str.replace("{'neg':","")
                                .str.strip()
                                .astype(float)
                               )

twitter2['SentScore_NEU'] = (twitter2['SentScore'].astype(str)
                                .str.split(",", n=2, expand=True)
                                .loc[:,1]
                                .str.replace("'neu':","")
                                .str.strip()
                                .astype(float)
                              )

twitter2['SentScore_POS'] = (twitter2['SentScore'].astype(str)
                                .str.rsplit(",", n=2, expand=True)
                                .loc[:,1]
                                .str.replace("'pos':","")
                                .str.strip()
                                .astype(float)
                               )

twitter2['SentScore_Compound'] = (twitter2['SentScore'].astype(str)
                                .str.split(",", n=4, expand=True)
                                .loc[:,3]
                                .str.replace("'compound':","")
                                .str.replace('}','')
                                .str.strip()
                                .astype(float)
                               )


In [ ]:
#Import packages for parts of speech tagging

from nltk import word_tokenize, pos_tag, pos_tag_sents
  
#Create a column that returns parts of speech & associated counts
twitter2['POS']=pos_tag_sents(twitter2['PostMessage7'].apply(word_tokenize).tolist())



In [ ]:
#Create columns to count the # of words for specific parts of speech
#Force 'POS' column to string
twitter2["POS_Str"]=twitter2["POS"].astype(str)


#Count the number of nouns, verbs, adjectives, etc 
#Create a new column for each part of speech counting the number of occurrences


#POS Abbreviation Meanings

#CC	coordinating conjunction
#CD	cardinal digit
#DT	determiner
#EX	existential there
#FW	foreign word
#IN	preposition/subordinating conjunction
#JJ	This NLTK POS Tag is an adjective (large)
#JJR	adjective, comparative (larger)
#JJS	adjective, superlative (largest)
#LS	list market
#MD	modal (could, will)
#NN	noun, singular (cat, tree)
#NNS	noun plural (desks)
#NNP	proper noun, singular (sarah)
#NNPS	proper noun, plural (indians or americans)
#PDT	predeterminer (all, both, half)
#POS	possessive ending (parent\ ‘s)
#PRP	personal pronoun (hers, herself, him, himself)
#PRP$	possessive pronoun (her, his, mine, my, our )
#RB	adverb (occasionally, swiftly)
#RBR	adverb, comparative (greater)
#RBS	adverb, superlative (biggest)
#RP	particle (about)
#TO	infinite marker (to)
#UH	interjection (goodbye)
#VB	verb (ask)
#VBG	verb gerund (judging)
#VBD	verb past tense (pleaded)
#VBN	verb past participle (reunified)
#VBP	verb, present tense not 3rd person singular(wrap)
#VBZ	verb, present tense with 3rd person singular (bases)
#WDT	wh-determiner (that, what)
#WP	wh- pronoun (who)
#WRB	wh- adverb (how)


twitter2["POS_Noun_Sing_Count"]=twitter2["POS_Str"].map(lambda x: x.count("'NN')"))

twitter2["POS_Noun_Plural_Count"]=twitter2["POS_Str"].map(lambda x: x.count("'NNS')"))

twitter2["POS_Noun_Proper_Sing_Count"]=twitter2["POS_Str"].map(lambda x: x.count("'NNP')"))

twitter2["POS_Noun_Proper_Plural_Count"]=twitter2["POS_Str"].map(lambda x: x.count("'NNPS')"))


twitter2["POS_Verb_Count"]=twitter2["POS_Str"].map(lambda x: x.count("'VB')"))

twitter2["POS_Verb_Gerund_Count"]=twitter2["POS_Str"].map(lambda x: x.count("'VBG')"))

twitter2["POS_Verb_PastTense_Count"]=twitter2["POS_Str"].map(lambda x: x.count("'VBD')"))

twitter2["POS_Verb_PastPartic_Count"]=twitter2["POS_Str"].map(lambda x: x.count("'VBN')"))

twitter2["POS_Verb_PT_Not3rdPerSingular_Count"]=twitter2["POS_Str"].map(lambda x: x.count("'VBP')"))

twitter2["POS_Verb_PT_With3rdPerSingular_Count"]=twitter2["POS_Str"].map(lambda x: x.count("'VBZ')"))


twitter2["POS_Adjective_Count"]=twitter2["POS_Str"].map(lambda x: x.count("'JJ')"))

twitter2["POS_Adjective_Comparative_Count"]=twitter2["POS_Str"].map(lambda x: x.count("'JJR')"))

twitter2["POS_Adjective_Superlative_Count"]=twitter2["POS_Str"].map(lambda x: x.count("'JJS')"))


twitter2["POS_Adverb_Count"]=twitter2["POS_Str"].map(lambda x: x.count("'RB')"))

twitter2["POS_Adverb_Comparative_Count"]=twitter2["POS_Str"].map(lambda x: x.count("'RBR')"))

twitter2["POS_Adverb_Superlative_Count"]=twitter2["POS_Str"].map(lambda x: x.count("'RBS')"))



twitter2["POS_Modal_Count"]=twitter2["POS_Str"].map(lambda x: x.count("'MD')"))

twitter2["POS_Cardinal_Digit_Count"]=twitter2["POS_Str"].map(lambda x: x.count("'CD')"))



twitter2["POS_Personal_Pronoun_Count"]=twitter2["POS_Str"].map(lambda x: x.count("'PRP')"))

twitter2["POS_Possessive_Pronoun_Count"]=twitter2["POS_Str"].map(lambda x: x.count("'PRP$')"))


twitter2["POS_Coord_Conj_Count"]=twitter2["POS_Str"].map(lambda x: x.count("'CC')"))

twitter2["POS_Determiner_Pronoun_Count"]=twitter2["POS_Str"].map(lambda x: x.count("'DT')"))



twitter2["POS_Preposition_SubordConj_Count"]=twitter2["POS_Str"].map(lambda x: x.count("'IN')"))

twitter2["POS_Predeterminer_Count"]=twitter2["POS_Str"].map(lambda x: x.count("'PDT')"))

twitter2["POS_Particle_Count"]=twitter2["POS_Str"].map(lambda x: x.count("'RP')"))



twitter2["POS_Interjection_Count"]=twitter2["POS_Str"].map(lambda x: x.count("'UH')"))

twitter2["POS_Possessive_End_Count"]=twitter2["POS_Str"].map(lambda x: x.count("'POS')"))

twitter2["POS_InfiniteMarker_End_Count"]=twitter2["POS_Str"].map(lambda x: x.count("'TO')"))



twitter2["POS_WH_Determiner_End_Count"]=twitter2["POS_Str"].map(lambda x: x.count("'WDT')"))

twitter2["POS_WH_Pronoun_End_Count"]=twitter2["POS_Str"].map(lambda x: x.count("'WP')"))

twitter2["POS_WH_Adverb_End_Count"]=twitter2["POS_Str"].map(lambda x: x.count("'WRB')"))






In [ ]:
#Consolidate parts of speech 

#Verbs
twitter2['Total_Verb_Count']= twitter2["POS_Verb_Count"] + twitter2["POS_Verb_Gerund_Count"]+ twitter2["POS_Verb_PastTense_Count"]+ twitter2["POS_Verb_PastPartic_Count"]+ twitter2["POS_Verb_PT_Not3rdPerSingular_Count"]+ twitter2["POS_Verb_PT_With3rdPerSingular_Count"]

#Adjectives 
twitter2['Total_Adjective_Count']=twitter2["POS_Adjective_Count"] + twitter2["POS_Adjective_Comparative_Count"] + twitter2["POS_Adjective_Superlative_Count"]

#Adverbs
twitter2['Total_Adverbs_Count']=twitter2["POS_Adverb_Count"] + twitter2["POS_Adverb_Comparative_Count"] + twitter2["POS_Adverb_Superlative_Count"]

#Noun Count
twitter2['Total_Noun_Count']=twitter2["POS_Noun_Sing_Count"] + twitter2["POS_Noun_Plural_Count"] + twitter2["POS_Noun_Proper_Sing_Count"] + twitter2["POS_Noun_Proper_Plural_Count"]



In [ ]:
#Create a column for word count

twitter2['Word_Count']= twitter2['PostMessage6'].str.split().map(len)

In [ ]:
#Create fields with % of words as nouns, verbs, adjectives, and adverbs

twitter2['Total_Verb_Perc_of_Words'] = twitter2['Total_Verb_Count'] / twitter2['Word_Count']

twitter2['Total_Adjective_Perc_of_Words'] = twitter2['Total_Adjective_Count'] / twitter2['Word_Count']

twitter2['Total_Adverb_Perc_of_Words'] = twitter2['Total_Adverbs_Count'] / twitter2['Word_Count']

twitter2['Total_Noun_Perc_of_Words'] = twitter2['Total_Noun_Count'] / twitter2['Word_Count']

twitter2['Total_Digit_Perc_of_Words']=twitter2['POS_Cardinal_Digit_Count']/ twitter2['Word_Count']


In [ ]:
# At this point, proprietary data is pulled into the workspace
# brand_dataset is a list of brand names and Twitter handles stored
# in Fanbase, as well as university athletics & non-athletics handles/names, 
# and professional sports handles/names

brand_dataset = Dataset.get_by_name(workspace, name='dimContent + dimBrand')

In [ ]:
#Convert to pandas dataframe
brands = brand_dataset.to_pandas_dataframe()

In [ ]:
#dataframe with Non University brand twitter handles 
brands_NonUniv = brands[~brands.unified_industry_name.isin(["University - Athletics", 
"University - Departments/Colleges",
"University - Alumni Association",
"Entertainment/Recreation - Major League Sports"])]

#dataframe with University Athletics handles 
brands_UnivAth = brands[brands.unified_industry_name.isin(["University - Athletics"])]

#dataframe with Other University handles 
brands_UnivOther = brands[brands.unified_industry_name.isin([ 
"University - Departments/Colleges",
"University - Alumni Association"])]

#dataframe with PRO SPORTS handles 
brands_ProSports = brands[brands.unified_industry_name.isin([ 
"Entertainment/Recreation - Major League Sports"])]


In [ ]:
#Subset brand dataframe to take a single columns (twitter handle)
#Create pattern for brand twitter handles (i.e. @delta)

brands_NonUniv['twitter_handle']=brands_NonUniv['twitter_handle'].astype(str)
brands_NonUniv['twitter_handle']=brands_NonUniv['twitter_handle'].str.lower()
brand_NonUniv_Handles_list = brands_NonUniv["twitter_handle"].values.tolist()
brand_NonUniv_Handles_list_pattern = '|'.join(brand_NonUniv_Handles_list)

In [ ]:
#Subset brand dataframe to take a single columns (twitter handle)
#Create pattern for university athletics handles (ie.e. @ou_athletics)
brands_UnivAth['twitter_handle']=brands_UnivAth['twitter_handle'].astype(str)
brands_UnivAth['twitter_handle']=brands_UnivAth['twitter_handle'].str.lower()
brand_UnivAth_Handles_list = brands_UnivAth["twitter_handle"].values.tolist()
brand_UnivAth_Handles_list_pattern = '|'.join(brand_UnivAth_Handles_list)

In [ ]:
#Subset brand dataframe to take a single columns (twitter handle)
#Create pattern for non-athletics university handles (i.e. @cuboulder)
brands_UnivOther['twitter_handle']=brands_UnivOther['twitter_handle'].astype(str)
brands_UnivOther['twitter_handle']=brands_UnivOther['twitter_handle'].str.lower()
brand_UnivOther_Handles_list = brands_UnivOther["twitter_handle"].values.tolist()
brand_UnivOther_Handles_list_pattern = '|'.join(brand_UnivOther_Handles_list)

In [ ]:
#Subset brand dataframe to take a single columns (twitter handle)
#Create pattern for pro sports handles (i.e. @broncos)
brands_ProSports['twitter_handle']=brands_ProSports['twitter_handle'].astype(str)
brands_ProSports['twitter_handle'].str.lower()
brand_ProSports_Handles_list = brands_ProSports["twitter_handle"].values.tolist()
brand_ProSports_Handles_list_pattern = '|'.join(brand_ProSports_Handles_list)

In [ ]:
#Subset brand dataframe to take a single columns (brand names)
#Create pattern for brand Names (i.e. ' best buy ')
brands_NonUniv['brand_name']=brands_NonUniv['brand_name'].astype(str)
brands_NonUniv['brand_name']=brands_NonUniv['brand_name'].str.lower()

brands_NonUniv['brand_name']=brands_NonUniv['brand_name'].str.replace('(', '')
brands_NonUniv['brand_name']=brands_NonUniv['brand_name'].str.replace(')', '')

#remove TV network brands (this is it's own field)
brands_NonUniv = brands_NonUniv.loc[brands_NonUniv['brand_name'] != 'abc']
brands_NonUniv = brands_NonUniv.loc[brands_NonUniv['brand_name'] != 'espn']
brands_NonUniv = brands_NonUniv.loc[brands_NonUniv['brand_name'] != 'espnu']
brands_NonUniv = brands_NonUniv.loc[brands_NonUniv['brand_name'] != 'espn2']
brands_NonUniv = brands_NonUniv.loc[brands_NonUniv['brand_name'] != 'espn3']
brands_NonUniv = brands_NonUniv.loc[brands_NonUniv['brand_name'] != 'fox']
brands_NonUniv = brands_NonUniv.loc[brands_NonUniv['brand_name'] != 'fs1']
brands_NonUniv = brands_NonUniv.loc[brands_NonUniv['brand_name'] != 'fs2']
brands_NonUniv = brands_NonUniv.loc[brands_NonUniv['brand_name'] != 'cbs']
brands_NonUniv = brands_NonUniv.loc[brands_NonUniv['brand_name'] != 'nbc']
brands_NonUniv = brands_NonUniv.loc[brands_NonUniv['brand_name'] != 'amazon prime']
brands_NonUniv = brands_NonUniv.loc[brands_NonUniv['brand_name'] != 'sec network']
brands_NonUniv = brands_NonUniv.loc[brands_NonUniv['brand_name'] != 'longhorn network']
brands_NonUniv = brands_NonUniv.loc[brands_NonUniv['brand_name'] != 'big ten network']
brands_NonUniv = brands_NonUniv.loc[brands_NonUniv['brand_name'] != 'byu tv']
brands_NonUniv = brands_NonUniv.loc[brands_NonUniv['brand_name'] != 'btn']
brands_NonUniv = brands_NonUniv.loc[brands_NonUniv['brand_name'] != 'acc network']
brands_NonUniv = brands_NonUniv.loc[brands_NonUniv['brand_name'] != 'pac 12 network']
brands_NonUniv = brands_NonUniv.loc[brands_NonUniv['brand_name'] != 'tbs']
brands_NonUniv = brands_NonUniv.loc[brands_NonUniv['brand_name'] != 'mountainwest network']

#Add space before and after brand name
brands_NonUniv['brand_name2'] = ' ' + brands_NonUniv['brand_name'] + ' '

brand_NonUniv_Names_list = brands_NonUniv['brand_name2'].values.tolist()
brand_NonUniv_Names_list_pattern = '|'.join(brand_NonUniv_Names_list)

In [ ]:
#Subset brand dataframe to take a single columns (brand names)
#Create pattern for college sports names (i.e. 'georgia bulldogs')
brands_UnivAth['brand_name']=brands_UnivAth['brand_name'].astype(str)
brands_UnivAth['brand_name']=brands_UnivAth['brand_name'].str.lower()

brands_UnivAth['brand_name']=brands_UnivAth['brand_name'].str.replace('(', '')
brands_UnivAth['brand_name']=brands_UnivAth['brand_name'].str.replace(')', '')

brand_UnivAth_Names_list = brands_UnivAth['brand_name'].values.tolist()
brand_UnivAth_Names_list_pattern = '|'.join(brand_UnivAth_Names_list)

In [ ]:
#Subset brand dataframe to take a single columns (brand names)
#Create pattern for university non-ath names (i.e. 'purdue university')
brands_UnivOther['brand_name']=brands_UnivOther['brand_name'].astype(str)
brands_UnivOther['brand_name']=brands_UnivOther['brand_name'].str.lower()

brands_UnivOther['brand_name']=brands_UnivOther['brand_name'].str.replace('(', '')
brands_UnivOther['brand_name']=brands_UnivOther['brand_name'].str.replace(')', '')

brand_UnivOther_Names_list = brands_UnivOther['brand_name'].values.tolist()
brand_UnivOther_Names_list_pattern = '|'.join(brand_UnivOther_Names_list)

In [ ]:
#Subset brand dataframe to take a single columns (brand names)
#Create pattern for pro sports names (i.e. 'atlanta hawks')
brands_ProSports['brand_name']=brands_ProSports['brand_name'].astype(str)
brands_ProSports['brand_name']=brands_ProSports['brand_name'].str.lower()

brands_ProSports['brand_name']=brands_ProSports['brand_name'].str.replace('(', '')
brands_ProSports['brand_name']=brands_ProSports['brand_name'].str.replace(')', '')
brands_ProSports['brand_name']=brands_ProSports['brand_name'].str.replace('x - ', '')
brands_ProSports['brand_name']=brands_ProSports['brand_name'].str.replace(' - x', '')

brand_ProSports_Names_list = brands_ProSports['brand_name'].values.tolist()
brand_ProSports_Names_list_pattern = '|'.join(brand_ProSports_Names_list)

In [ ]:
#Create indicators columns to flag if handles/names of brands, college athletics
#college non-athletics, and pro sports show up in Twitter content

#Create flags to determine if tweet contains brand handles, names
twitter2['Word_Brand_Handles_Flag'] = twitter2['PostMessage6'].str.contains(brand_NonUniv_Handles_list_pattern).astype(int)
twitter2['Word_Brand_Names_Flag'] = twitter2['PostMessage6'].str.contains(brand_NonUniv_Names_list_pattern).astype(int)

twitter2['Word_UnivAth_Handles_Flag'] = twitter2['PostMessage6'].str.contains(brand_UnivAth_Handles_list_pattern).astype(int)

twitter2['Word_UnivOther_Handles_Flag'] = twitter2['PostMessage6'].str.contains(brand_UnivOther_Handles_list_pattern).astype(int)

twitter2['Word_ProSports_Handles_Flag'] = twitter2['PostMessage6'].str.contains(brand_ProSports_Handles_list_pattern).astype(int)
twitter2['Word_ProSports_Names_Flag'] = twitter2['PostMessage6'].str.contains(brand_ProSports_Names_list_pattern).astype(int)






In [ ]:
# At this point, proprietary data is pulled into the workspace
# venue_dataset is a list of college athletics venues and
# associated city names

venue_dataset = Dataset.get_by_name(workspace, name='dimVenue')

In [ ]:
#Convert to pandas dataframe
venues = venue_dataset.to_pandas_dataframe()

In [ ]:
#Create pattern of college sports venue names (i.e. Kinnick Stadium) show up
venues['venue_name']=venues['venue_name'].astype(str)
venues['venue_name']=venues['venue_name'].str.lower()

venues['venue_name']=venues['venue_name'].str.replace('(', '')
venues['venue_name']=venues['venue_name'].str.replace(')', '')
venues['venue_name']=venues['venue_name'].str.replace('x - ', '')
venues['venue_name']=venues['venue_name'].str.replace(' - x', '')

venue_name_list = venues['venue_name'].values.tolist()
venue_name_list_pattern = '|'.join(venue_name_list)

In [ ]:
#Create pattern of college sports city names (i.e. Iowa City) show up
venues['city']=venues['city'].astype(str)
venues['city']=venues['city'].str.lower()

venues['city']=venues['city'].str.replace('(', '')
venues['city']=venues['city'].str.replace(')', '')
venues['city']=venues['city'].str.replace('x - ', '')
venues['city']=venues['city'].str.replace(' - x', '')

city_name_list = venues['city'].values.tolist()
city_name_list_pattern = '|'.join(city_name_list)

In [ ]:
#Indicator columns to flag whether keywords from college sports venue and city list appear in tweet
twitter2['Word_Venue_List_Flag'] = twitter2['PostMessage6'].str.contains(venue_name_list_pattern).astype(int)

twitter2['Word_City_List_Flag'] = twitter2['PostMessage6'].str.contains(city_name_list_pattern).astype(int)

In [ ]:
#Indicator column: does Tweet contain "team name" (i.e. hawkeyes, sooners)
#Import Team Names list for lookup

teamnames = pd.read_csv('TeamName_Lookup_MITSloan.csv')
teamnames["TeamNames"]=teamnames["TeamNames"].astype(str)
teamnames["TeamNames"]=teamnames["TeamNames"].str.lower()

#Convert to Team Names to list
teamnames_list = teamnames["TeamNames"].values.tolist()

#Ad 'Or' operator to team names list -- use this pattern later in str.contains
team_list_pattern = '|'.join(teamnames_list)

#Create field -- inidcator columns for whether mascot shows up
twitter2['Word_TeamName_Flag'] = twitter2['PostMessage6'].str.contains(team_list_pattern).astype(int)



In [ ]:
#Create indicator columns for whether certain keywords (that don't require Fanbase integration)
# appear in tweets

#Birthday Flag
birthday_list = ['birthday', 'hbd']
birthday_list_pattern = '|'.join(birthday_list)
twitter2['Word_Birthday_Flag'] = twitter2['PostMessage6'].str.contains(birthday_list_pattern).astype(int)

#Conference Flag 
conference_list = ['b1g', 'big ten', 
                'pac12', 'pac 12', 'pac-12', 'pac twelve'
                'big 12', 'big twelve', 'big12',
                ' sec ', 
                ' acc ',  
                ' aac ','american athletic conference',
                'mountain west', 'mwc',
                'sunbelt',
                'c-usa','conference usa',
                'mid american conference',
                ' mac ']  
conference_list_pattern = '|'.join(conference_list)
twitter2['Word_Conference_Flag'] = twitter2['PostMessage6'].str.contains(conference_list_pattern).astype(int)



#TV Networks
tvnetworks_list = ['abc','nbc',' fox ','fs1','fs1','espn', 'cbs',
                'amazon prime', 'byu tv',
                'espnu','espn2','big ten network','btn',
                'sec network','acc network','pac 12 network',
                'tbs','longhorn network','mountainwest network']
                
tvnetworks_list_pattern = '|'.join(tvnetworks_list)
twitter2['Word_TVNetworks_Flag'] = twitter2['PostMessage6'].str.contains(tvnetworks_list_pattern).astype(int)



#AM_PM Flag 
ampm_list = ['a.m.','p.m.','12 pm', '12pm', 'noon',
                '1 pm','2 pm','3 pm','4 pm','5 pm','6 pm',
                '7 pm','8 pm','9 pm', '10 pm', '11 pm',
                '1 am','2 am','3 am','4 am','5 am','6 am',
                '7 am','8 am','9 am', '10 am', '11 am',
                 '1pm','2pm','3pm','4pm','5pm','6pm',
                '7pm','8pm','9pm', '10pm', '11pm',
                '1am','2am','3am','4am','5am','6am',
                '7am','8am','9am', '10am', '11am',
                ]            
ampm_list_pattern = '|'.join(ampm_list)
twitter2['Word_ampm_Flag'] = twitter2['PostMessage6'].str.contains(ampm_list_pattern).astype(int)


#Ticket 
ticket_list = ['ticket']
                
ticket_list_pattern = '|'.join(ticket_list)
twitter2['Word_Ticket_Flag'] = twitter2['PostMessage6'].str.contains(ticket_list_pattern).astype(int)


#Congratulations
congrats_list = ['congrat']
                
congrats_list_pattern = '|'.join(congrats_list)
twitter2['Word_Congrats_Flag'] = twitter2['PostMessage6'].str.contains(congrats_list_pattern).astype(int)


#Holidays 
holiday_list = ['holiday', 'labor day',
                'halloween', 'veterans day','veteran''s day',
                'thanksgiving','christmas', 'new year',
                'martin luther king jr day', 
                'martin luther king jr. day',
                'mlk day', 'groundhog day',
                'president''s day','presidents day',
                'mardi gras',
                'valentine','st. patrick','saint patrick',
                 'easter', 'indigenous peoples day',
                 'mother''s day','mothers day',
                 'father''s day','fathers day',
                'memorial day','cinco de mayo'
                'juneteenth','flag day',
                'fouth of july','4th of july','july 4th',
                'happy fourth','happy 4th',
                'july 4','independence day','black friday',
                'patriots day','patriot''s day',
                'kwanzaa','hanukkah','ramadan'
                ]
                
holiday_list_pattern = '|'.join(holiday_list)
twitter2['Word_Holiday_Flag'] = twitter2['PostMessage6'].str.contains(holiday_list_pattern).astype(int)


#Miliatry
military_list = ['military','troops','veteran']               
military_list_pattern = '|'.join(military_list)
twitter2['Word_Military_Flag'] = twitter2['PostMessage6'].str.contains(military_list_pattern).astype(int)


#Day of Week
dayofweek_list = ['monday','tuesday','wednesday','thursday','friday','saturday',
                    'sunday']               
dayofweek_list_pattern = '|'.join(dayofweek_list)
twitter2['Word_DayofWeek_Flag'] = twitter2['PostMessage6'].str.contains(dayofweek_list_pattern).astype(int)


#Month
month_list = ['january','february','march','april',' may ','june','july',
                    'august','september','october','november','december']              
month_list_pattern = '|'.join(month_list)
twitter2['Word_Month_Flag'] = twitter2['PostMessage6'].str.contains(month_list_pattern).astype(int)


#Year - 1900s
year1900s_list = ['1900','1901','1902','1903','1904','1905','1906',
                    '1907','1908','1909','1910','1911','1912','1913','1914','1915','1916',
                    '1917','1918','1919','20s','1920','1921','1922','1923','1924','1925','1926',
                    '1927','1928','1929','30s','1930','1931','1932','1933','1934','1935','1936',
                    '1937','1938','1939','40s','1940','1941','1942','1943','1944','1945','1946',
                    '1947','1948','1949','50s','1950','1951','1952','1953','1954','1955','1956',
                    '1957','1958','1959','60s','1960','1961','1962','1963','1964','1965','1966',
                    '1967','1968','1969','70s','1970','1971','1972','1973','1974','1975','1976',
                    '1977','1978','1979','80s','1980','1981','1982','1983','1984','1985','1986',
                    '1987','1988','1989','90s','1990','1991','1992','1993','1994','1995','1996',
                    '1997','1998','1999']
year1900s_list_pattern = '|'.join(year1900s_list)
twitter2['Word_Year1900s_Flag'] = twitter2['PostMessage6'].str.contains(year1900s_list_pattern).astype(int)


#2000s
Year2000s_list = ['2000','2001','2002','2003','2004','2005','2006',
                    '2007','2008','2009']               
Year2000s_list_pattern = '|'.join(Year2000s_list)
twitter2['Word_Year2000s_Flag'] = twitter2['PostMessage6'].str.contains(Year2000s_list_pattern).astype(int)


#2010s
Year2010s_list = ['2010','2011','2012','2013','2014','2015','2016',
                    '2017','2018','2019']               
Year2010s_list_pattern = '|'.join(Year2010s_list)
twitter2['Word_Year2010s_Flag'] = twitter2['PostMessage6'].str.contains(Year2010s_list_pattern).astype(int)


#2020-22
Year2020_22_list = ['2020','2021','2022']
Year2020_22_list_pattern = '|'.join(Year2020_22_list)
twitter2['Word_Year2020_22_Flag'] = twitter2['PostMessage6'].str.contains(Year2020_22_list_pattern).astype(int)


#2023-29
Year2023_29_list = ['2023','2024','2025', '2026','2027','2028','2029']
Year2023_29_list_pattern = '|'.join(Year2023_29_list)
twitter2['Word_Year2023_29_Flag'] = twitter2['PostMessage6'].str.contains(Year2023_29_list_pattern).astype(int)


#coach
coach_list = ['coach']
coach_list_pattern = '|'.join(coach_list)
twitter2['Word_Coach_Flag'] = twitter2['PostMessage6'].str.contains(coach_list_pattern).astype(int)


#Win
win_list = [' win ','victory','winning',' won ', 'champions']
win_list_pattern = '|'.join(win_list)
twitter2['Word_Win_Flag'] = twitter2['PostMessage6'].str.contains(win_list_pattern).astype(int)


#Rank 
rank_list = ['ranking','ranked',' rank ']
rank_list_pattern = '|'.join(rank_list)
twitter2['Word_Rank_Flag'] = twitter2['PostMessage6'].str.contains(rank_list_pattern).astype(int)


#Pro Sports Leagues
proleague_list = ['nfl','nba','nhl','mlb','wnba','pga','cfl']
proleague_list_pattern = '|'.join(proleague_list)
twitter2['Word_ProLeague_Flag'] = twitter2['PostMessage6'].str.contains(proleague_list_pattern).astype(int)


#Award
award_list = ['award','of the year','of the year']
award_list_pattern = '|'.join(award_list)
twitter2['Word_Award_Flag'] = twitter2['PostMessage6'].str.contains(award_list_pattern).astype(int)


#Academics
academics_list = ['academic','classroom','gpa']
academics_list_pattern = '|'.join(academics_list)
twitter2['Word_Academics_Flag'] = twitter2['PostMessage6'].str.contains(academics_list_pattern).astype(int)


#Donation
donation_list = ['donation']
donation_list_pattern = '|'.join(donation_list)
twitter2['Word_Donation_Flag'] = twitter2['PostMessage6'].str.contains(donation_list_pattern).astype(int)


#GameStats
gamestats_list = ['touchdown','interception','completion',
                    'field goal','extra point','safety',
                    'penalty','yard','rush','sack','tackle',
                    'time out','fumble','reception','pass',
                    'hurries','touchback','punt','snap',
                    'td','fg', '1st down', 'first down',
                    '2nd down','second down','3rd down',
                    'third down','4th down','fourth down',
                    'turnover',
                    'first half','1st half',
                    'second half','2nd half',

                    '1Q','Q1','2Q','Q2','3Q','Q3','4Q','Q4',

                    'rebound','assist','free throw',
                    '3 pointer','3 pt', 
                    'three pointer','three-pointer','3-pointer',
                    '3 points', 
                    'foul', 'block', 'steal','double double',
                    'triple double',

                    'point',

                    'inning','hits','rbi','stikeout','runs','error','home run'

                    'kills', 'ace', 'attempt']

gamestats_list_pattern = '|'.join(gamestats_list)
twitter2['Word_GameStats_Flag'] = twitter2['PostMessage6'].str.contains(gamestats_list_pattern).astype(int)


#Time (Generic)
time_generic_list = ['today','tomorrow','yesterday','weekend','week','tonight',
                    'afternoon','morning','night','evening']
time_generic_list_pattern = '|'.join(time_generic_list)
twitter2['Word_Time_Generic_Flag'] = twitter2['PostMessage6'].str.contains(time_generic_list_pattern).astype(int)


#Position
position_list = ['quarterback','quarter back','QB1','QB',
                 'running back','runningback',
                    'tight end',
                    'receiver',
                    'half back', 'halfback',
                    'full back', 'fullback',
                    'right tackle','right guard','center','left guard','left tackle',
                    'offensive line','o line',
                    'defensive line','d line',
                    'd tackle','defensive tackle',
                    'defensive end','d end', 'nose guard','nose tackle',
                    'linebacker',
                    'nickleback',
                    'corner','cb','d back','defensive back',
                    'safety', 
                    'kicker','punter','snapper','returner',

                    'point guard','shooting guard','power forward','small forward',

                    'pitcher','catcher','first base','1st base','second base',
                    '2nd base','third base','3rd base','outfield','short stop',

                    'hitter', 'setter', 'middle blocker', 'libero', 
                    'specialist']

position_list_pattern = '|'.join(position_list)
twitter2['Word_Position_Flag'] = twitter2['PostMessage6'].str.contains(position_list_pattern).astype(int)




#GenZ Terms
genz_terms_list = ['vibe','vibing',
                    'hits different','hits dif'
                    ' drip ',
                    ' no cap ',
                    'main character',
                    'understood the assignment','understand the assignment', 'understands the assignment'
                    ' slaps '
                    'low key','low-key', 'lowkey',
                    'high key','high-key', 'highkey',
                    'bussin',
                    'glow up',
                    'big yikes', 
                    'finna',
                    'sip tea',
                    'yeet',
                    'savage',
                     'iykyk',
                     ' flex ',
                     ' lit ',
                     'finesse',
                     'savage']
genz_terms_list_pattern = '|'.join(genz_terms_list)
twitter2['Word_GenZ_Terms_Flag'] = twitter2['PostMessage6'].str.contains(genz_terms_list_pattern).astype(int)


#Venue 
venue_list = ['stadium','arena','dome','court']
venue_list_pattern = '|'.join(venue_list)
twitter2['Word_Venue_Flag'] = twitter2['PostMessage6'].str.contains(venue_list_pattern).astype(int)


#Schedule 
schedule_list = ['schedule']
schedule_list_pattern = '|'.join(schedule_list)
twitter2['Word_Schedule_Flag'] = twitter2['PostMessage6'].str.contains(schedule_list_pattern).astype(int)


#Colors
color_list = [' red ', 
            'blue','yellow','gold','black',
                'white','green','purple','crimson','scarlet',
                        'orange','maroon','silver','cream',
                        'brown','cardinal','grey']
color_list_pattern = '|'.join(color_list)
twitter2['Word_Color_Flag'] = twitter2['PostMessage6'].str.contains(color_list_pattern).astype(int)

#Free
free_list = ['free','giveaway']
free_list_pattern = '|'.join(free_list)
twitter2['Word_Free_Flag'] = twitter2['PostMessage6'].str.contains(free_list_pattern).astype(int)

#Presented By
presentedby_list = ['presented by']
presentedby_list_pattern = '|'.join(presentedby_list)
twitter2['Word_PresentedBy_Flag'] = twitter2['PostMessage6'].str.contains(presentedby_list_pattern).astype(int)

#Ready To
ready_list = ['ready']
ready_list_pattern = '|'.join(ready_list)
twitter2['Word_Ready_Flag'] = twitter2['PostMessage6'].str.contains(ready_list_pattern).astype(int)

#Throwback
throwback_list = ['throwback','flashback','tbt']
throwback_list_pattern = '|'.join(throwback_list)
twitter2['Word_Throwback_Flag'] = twitter2['PostMessage6'].str.contains(throwback_list_pattern).astype(int)

#Thanks
thank_list = ['thank']
thank_list_pattern = '|'.join(thank_list)
twitter2['Word_Thanks_Flag'] = twitter2['PostMessage6'].str.contains(thank_list_pattern).astype(int)

#Good luck
goodluck_list = ['good luck']
goodluck_list_pattern = '|'.join(goodluck_list)
twitter2['Word_GoodLuck_Flag'] = twitter2['PostMessage6'].str.contains(goodluck_list_pattern).astype(int)

#Celebrate
celebrate_list = ['celebrate']
celebrate_list_pattern = '|'.join(celebrate_list)
twitter2['Word_Celebrate_Flag'] = twitter2['PostMessage6'].str.contains(celebrate_list_pattern).astype(int)

#Be sure
be_sure_list = ['be sure']
be_sure_list_pattern = '|'.join(be_sure_list)
twitter2['Word_Be_Sure_Flag'] = twitter2['PostMessage6'].str.contains(be_sure_list_pattern).astype(int)

#Against vs. 
against_list = ['against','vs']
against_list_pattern = '|'.join(against_list)
twitter2['Word_Against_Flag'] = twitter2['PostMessage6'].str.contains(against_list_pattern).astype(int)

#Game
game_list = ['game']
game_list_pattern = '|'.join(game_list)
twitter2['Word_Game_Flag'] = twitter2['PostMessage6'].str.contains(game_list_pattern).astype(int)

#Team
team_list = ['team']
team_list_pattern = '|'.join(team_list)
twitter2['Word_Team_Flag'] = twitter2['PostMessage6'].str.contains(team_list_pattern).astype(int)

#Back to
back_list = ['back to','back at']
back_list_pattern = '|'.join(back_list)
twitter2['Word_Back_To_Flag'] = twitter2['PostMessage6'].str.contains(back_list_pattern).astype(int)

#Sport
sport_list = ['football','basketball','mbb','wbb','baseball','softball',
            'hockey','soccer','tennis','golf','rowing','volleyball',
            'cross country','swim','dive','track','wrestling','gymnastic',
            'lacrosse','water polo']
sport_list_pattern = '|'.join(sport_list)
twitter2['Word_Sport_Flag'] = twitter2['PostMessage6'].str.contains(sport_list_pattern).astype(int)


In [ ]:
#Drop any tweets with 0 impressions to avoid NaN share rates
twitter3 = twitter2.drop(twitter2[twitter2.Impressions <1].index)
twitter3 = twitter3.drop(twitter3[twitter3.Impressions.isnull()].index)
twitter3 = twitter3.drop(twitter3[twitter3.Impressions.isna()].index)
twitter3 = twitter3.drop(twitter3[twitter3.Impressions==''].index)


In [ ]:
#Simplify Dataset (reduce columns)

Twitter_Simple = twitter3[['Handle',

'VideoFlag',

'Fri_to_Sunday_Flag',

'Sport_Football',
'Sport_Main Athletics',
'Sport_Men\'s Basketball',
'Sport_Women\'s Basketball',
'Sport_Olympic',

'P5_Conference_Flag',

'Month_Sep_Oct',
'Month_Nov_Dec',
'Month_Jan_Feb',
'Month_Mar_Apr',
'Month_May_June',
'Month_July_Aug',

'Exlaim_Mark_Count',
'Question_Mark_Count',
'Hashtag_Count',
'Mention_Count',
'Line_Break_Count',
'Link_Count_v2',
'Word_Count',
'Emoji_Count',

'SentScore_Compound',

'Total_Verb_Perc_of_Words',
'Total_Adjective_Perc_of_Words',
'Total_Adverb_Perc_of_Words',
'Total_Noun_Perc_of_Words',
'Total_Digit_Perc_of_Words',

'Word_GenZ_Terms_Flag',
'Word_TeamName_Flag',
'Word_Time_Generic_Flag',
'Word_GameStats_Flag',
'Word_City_List_Flag',
'Word_Sport_Flag',
'Word_ampm_Flag',
'Word_Game_Flag',
'Word_UnivAth_Handles_Flag',
'Word_Win_Flag',
'Word_Coach_Flag',
'Word_Brand_Handles_Flag',
'Word_Position_Flag',
'Word_UnivOther_Handles_Flag',
'Word_Ticket_Flag',
'Word_ProLeague_Flag',
'Word_ProSports_Names_Flag',
'Word_Thanks_Flag',
'Word_Free_Flag',
'Word_Award_Flag',
'Word_Rank_Flag',
'Word_Birthday_Flag',
'Word_TVNetworks_Flag',
'Word_PresentedBy_Flag',
'Word_Year1900s_Flag',
'Word_Year2000s_Flag',
'Word_Year2010s_Flag',

'Emo_Sports_Objects_Flag',
'Emo_heavyexclaimation_Flag',
'Emo_animal_Flag',
'Emo_TV_Radio_Flag',
'Emo_arrows_Flag',
'Emo_numbers_Flag',
'Emo_circle_Flag',
'Emo_Bar_chart_Flag',
'Emo_eyes_Flag',
'Emo_newspaper_Flag',
'Emo_muscle_Flag',
'Emo_Round_Pushpin_Flag',
'Emo_rotating_light_Flag',
'Emo_link_Flag',
'Emo_fire_Flag',
'Emo_heart_Flag',
'Emo_triumph_Flag',
'Emo_handgesture_Flag',
'Emo_vs_Flag',
'Emo_trophyring_Flag',


'Impressions',
'VideoViews',
'Likes',
'Comments',
'Shares',
'Share_Rate',
'Comment_Rate',
'Like_Rate',
'Total_SCLs',
'SCL_Rate'

]]

In [ ]:
Twitter_Simple.rename({'Sport_Men\'s Basketball': 'Sport_MBB', 
                            'Sport_Women\'s Basketball': 'Sport_WBB',
                            'Sport_Main Athletics':'Sport_Main_Athletics'}, axis=1, inplace=True)

In [ ]:
#Import datframe containing Wiland demographics (data is proprietary, so file is not provided)
#Contains brekdown of age of followers for given handles

all_wiland = pd.read_csv('wiland_demos_MIT_Sloan_82922.csv')
all_wiland.head()

In [ ]:
#Consolidate Wiland Dataset
#Create Wiland field for 18 to 25

all_wiland['Wiland_Age_18_25']= all_wiland['wiland_age_18_20']+all_wiland['wiland_age_21_25']
all_wiland['Wiland_Age_26_35']= all_wiland['wiland_age_26_30']+all_wiland['wiland_age_31_35']
all_wiland['Wiland_Age_36_50']= all_wiland['wiland_age_36_40']+all_wiland['wiland_age_41_45']+all_wiland['wiland_age_46_50']
all_wiland['Wiland_Age_Over50']= all_wiland['wiland_age_51_55']+all_wiland['wiland_age_56_60']+all_wiland['wiland_age_61_65']+all_wiland['wiland_age_66_70']+all_wiland['wiland_age_71_75']+all_wiland['wiland_age_76+']

all_wiland['Wiland_Income_Over100k']=all_wiland['wiland_income_100K_124K']+all_wiland['wiland_income_125Kplus']

all_wiland['wiland_urbanicity_rural_consol']=all_wiland['wiland_urbanicity_Highly_Rural_Community']+all_wiland['wiland_urbanicity_Rural_Community']


In [ ]:
#Simplify wiland columns

all_wiland2 = all_wiland[['Handle_Key',
'Wiland_Age_18_25',
'Wiland_Age_26_35',
'Wiland_Age_36_50',
'Wiland_Age_Over50',

'Wiland_Income_Over100k',

'wiland_urbanicity_rural_consol',
'wiland_urbanicity_Urban_Community',
'wiland_urbanicity_Suburban_Community',

'wiland_eth_Asian',
'wiland_eth_African_American',
'wiland_eth_Hispanic',
'wiland_eth_White',

'wiland_hh_comp_Married',
'wiland_hh_comp_Children_Present'

]]

In [ ]:
#Join Wiland data with larger dataframe of Twitter data

Tw_Gran_Wiland = pd.merge(Twitter_Simple,all_wiland2,
            left_on='Handle',
            right_on='Handle_Key',
            how='left',
            left_index=False)

In [ ]:
#Drop Rows with NA values for Wiland Age demos

Tw_Gran_Wiland_FILT = Tw_Gran_Wiland.drop(Tw_Gran_Wiland[Tw_Gran_Wiland.Wiland_Age_18_25.isnull()].index)

Tw_Gran_Wiland_FILT = Tw_Gran_Wiland_FILT.drop(Tw_Gran_Wiland_FILT[Tw_Gran_Wiland_FILT.Wiland_Age_18_25.isna()].index)

Tw_Gran_Wiland_FILT = Tw_Gran_Wiland_FILT.drop(Tw_Gran_Wiland_FILT[Tw_Gran_Wiland_FILT.Wiland_Age_18_25==''].index)


In [ ]:
#Predicting Engagement at tweet level -- include Wiland fields
#Features were selected based on frequencies, relevance

Tw_Gran_Wiland_Model = Tw_Gran_Wiland_FILT[[
'Handle', #Used to identify subset of handles w/ highest and lowest % of GenZ followers (18-25) -- drop for modeling

'VideoFlag',
'Fri_to_Sunday_Flag',
'P5_Conference_Flag',
'Month_Sep_Oct',
'Month_Nov_Dec',
'Month_Jan_Feb',
'Month_Mar_Apr',
'Month_July_Aug',
'Exclaim_Mark_Count',
'Question_Mark_Count',
'Hashtag_Count',
'Mention_Count',
'Line_Break_Count',
'Link_Count_v2',
'Word_Count',
'Emoji_Count',
'SentScore_Compound',
'Total_Verb_Perc_of_Words',
'Total_Adjective_Perc_of_Words',
'Total_Adverb_Perc_of_Words',
'Total_Noun_Perc_of_Words',
'Total_Digit_Perc_of_Words',
'Word_GenZ_Terms_Flag',
'Word_TeamName_Flag',
'Word_Time_Generic_Flag',
'Word_GameStats_Flag',
'Word_City_List_Flag',
'Word_Sport_Flag',
'Word_ampm_Flag',
'Word_Game_Flag',
'Word_UnivAth_Handles_Flag',
'Word_Win_Flag',
'Word_Coach_Flag',
'Word_Brand_Handles_Flag',
'Word_Position_Flag',
'Word_UnivOther_Handles_Flag',
'Word_Ticket_Flag',
'Word_ProLeague_Flag',
'Word_ProSports_Names_Flag',
'Word_Thanks_Flag',
'Word_Free_Flag',
'Word_Award_Flag',
'Word_Rank_Flag',
'Word_Birthday_Flag',
'Word_TVNetworks_Flag',
'Word_PresentedBy_Flag',
'Word_Year1900s_Flag',
'Word_Year2000s_Flag',
'Word_Year2010s_Flag',
'Emo_Sports_Objects_Flag',
'Emo_heavyexclaimation_Flag',
'Emo_animal_Flag',
'Emo_TV_Radio_Flag',
'Emo_arrows_Flag',
'Emo_numbers_Flag',
'Emo_circle_Flag',
'Emo_Bar_chart_Flag',
'Emo_eyes_Flag',
'Emo_newspaper_Flag',
'Emo_muscle_Flag',
'Emo_Round_Pushpin_Flag',
'Emo_rotating_light_Flag',
'Emo_link_Flag',
'Emo_fire_Flag',
'Emo_heart_Flag',
'Emo_triumph_Flag',
'Emo_handgesture_Flag',
'Emo_vs_Flag',
'Emo_trophyring_Flag',

'Wiland_Age_18_25', #Age 18-25 % to be used to identify accounts with highest and lowest % of Gen Z

'Share_Rate', #Share Rate (Retweet Rate) to be used for Y (dependent) variable
'Comment_Rate',
'Like_Rate',
'SCL_Rate'

]]


In [ ]:
#Convert NaN Part of Speech % fields to 0
Tw_Gran_Wiland_Model['Total_Verb_Perc_of_Words'] = Tw_Gran_Wiland_Model['Total_Verb_Perc_of_Words'].fillna(0)
Tw_Gran_Wiland_Model['Total_Adjective_Perc_of_Words'] = Tw_Gran_Wiland_Model['Total_Adjective_Perc_of_Words'].fillna(0)
Tw_Gran_Wiland_Model['Total_Adverb_Perc_of_Words'] = Tw_Gran_Wiland_Model['Total_Adverb_Perc_of_Words'].fillna(0)
Tw_Gran_Wiland_Model['Total_Noun_Perc_of_Words'] = Tw_Gran_Wiland_Model['Total_Noun_Perc_of_Words'].fillna(0)
Tw_Gran_Wiland_Model['Total_Digit_Perc_of_Words'] = Tw_Gran_Wiland_Model['Total_Digit_Perc_of_Words'].fillna(0)

In [ ]:
#End Feature Engineering Code